In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from statsmodels.stats.power import NormalIndPower
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

def overview(df, n=5):
    print('\nShape:', df.shape)
    display(df.head(n))
    print('\nInfo:')
    display(df.info())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Edit this path if you mounted Drive or used upload. If you used files.upload(), set DATA_PATH to the uploaded filename.
DATA_PATH = '/content/drive/MyDrive/ecommerce.csv'  # <- edit if needed

def load_csv(path):
    try:
        return pd.read_csv(path, encoding='ISO-8859-1')
    except Exception:
        return pd.read_csv(path)

# Try load
if os.path.exists(DATA_PATH):
    df = load_csv(DATA_PATH)
    print('Loaded from', DATA_PATH)
else:
    # If you used files.upload(), find the file in the working directory (like 'ecommerce.csv')
    if os.path.exists('ecommerce.csv'):
        df = load_csv('ecommerce.csv')
        print('Loaded ecommerce.csv from working directory.')
    else:
        raise FileNotFoundError("Couldn't find the dataset. Upload via files.upload() or set DATA_PATH to the Drive path.")

overview(df)


Loaded from /content/drive/MyDrive/ecommerce.csv

Shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


None

In [ ]:
# Basic cleaning
df.drop_duplicates(inplace=True)
df.columns = [c.strip() for c in df.columns]

# Parse InvoiceDate
if 'InvoiceDate' in df.columns:
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
else:
    raise ValueError('InvoiceDate column not found. Check dataset columns: ' + ','.join(df.columns))

# Numeric conversions
for col in ['Quantity','UnitPrice']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Keep only rows with CustomerID (or remove this if you want anonymous traffic too)
if 'CustomerID' in df.columns:
    df = df[df['CustomerID'].notnull()].copy()
    try:
        df['CustomerID'] = df['CustomerID'].astype(int)
    except Exception:
        df['CustomerID'] = df['CustomerID'].astype(str)

# TotalAmount & returns flag
if all(c in df.columns for c in ['Quantity','UnitPrice']):
    df['TotalAmount'] = df['Quantity'] * df['UnitPrice']
df['IsReturn'] = (df.get('Quantity',0) < 0) | (df.get('TotalAmount',0) < 0)

# Date parts
df['InvoiceYearMonth'] = df['InvoiceDate'].dt.to_period('M')
df['InvoiceDateOnly'] = df['InvoiceDate'].dt.date
df['Weekday'] = df['InvoiceDate'].dt.day_name()

# Remove zero-value transactions (if any)
df = df[df['TotalAmount'] != 0].copy()

print("After cleaning:")
overview(df)


After cleaning:

Shape: (401564, 13)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount,IsReturn,InvoiceYearMonth,InvoiceDateOnly,Weekday
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,False,2010-12,2010-12-01,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,False,2010-12,2010-12-01,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,False,2010-12,2010-12-01,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,False,2010-12,2010-12-01,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,False,2010-12,2010-12-01,Wednesday



Info:
<class 'pandas.core.frame.DataFrame'>
Index: 401564 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         401564 non-null  object        
 1   StockCode         401564 non-null  object        
 2   Description       401564 non-null  object        
 3   Quantity          401564 non-null  int64         
 4   InvoiceDate       401564 non-null  datetime64[ns]
 5   UnitPrice         401564 non-null  float64       
 6   CustomerID        401564 non-null  int64         
 7   Country           401564 non-null  object        
 8   TotalAmount       401564 non-null  float64       
 9   IsReturn          401564 non-null  bool          
 10  InvoiceYearMonth  401564 non-null  period[M]     
 11  InvoiceDateOnly   401564 non-null  object        
 12  Weekday           401564 non-null  object        
dtypes: bool(1), datetime64[ns](1), float64(2), int64(2), obje

None

In [ ]:
# Monthly revenue and orders
revenue_m = df.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['TotalAmount'].sum().reset_index()
orders_m = df.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['InvoiceNo'].nunique().reset_index().rename(columns={'InvoiceNo':'Orders'})

fig = px.line(revenue_m, x='InvoiceDate', y='TotalAmount', title='Monthly Revenue')
fig.show()

fig2 = px.line(orders_m, x='InvoiceDate', y='Orders', title='Monthly Orders')
fig2.show()

# Top products by revenue (if Description exists)
if 'Description' in df.columns:
    top_products = df.groupby('Description')['TotalAmount'].sum().sort_values(ascending=False).head(10)
    print('\nTop 10 products by revenue:')
    display(top_products)


/tmp/ipython-input-2818303743.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  revenue_m = df.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['TotalAmount'].sum().reset_index()
/tmp/ipython-input-2818303743.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  orders_m = df.groupby(pd.Grouper(key='InvoiceDate', freq='M'))['InvoiceNo'].nunique().reset_index().rename(columns={'InvoiceNo':'Orders'})



Top 10 products by revenue:


,TotalAmount
Description,
REGENCY CAKESTAND 3 TIER,132567.70
WHITE HANGING HEART T-LIGHT HOLDER,93767.80
JUMBO BAG RED RETROSPOT,83056.52
PARTY BUNTING,67628.43
POSTAGE,66710.24
ASSORTED COLOUR BIRD ORNAMENT,56331.91
RABBIT NIGHT LIGHT,51042.84
CHILLI LIGHTS,45915.41
PAPER CHAIN KIT 50'S CHRISTMAS,41423.78


In [ ]:
# Build cohort months
cust_first = df.groupby('CustomerID')['InvoiceDate'].min().reset_index().rename(columns={'InvoiceDate':'FirstOrder'})
cust_first['CohortMonth'] = cust_first['FirstOrder'].dt.to_period('M')

df = df.merge(cust_first[['CustomerID','CohortMonth']], on='CustomerID', how='left')
df['OrderMonth'] = df['InvoiceDate'].dt.to_period('M')

cohort = df.groupby(['CohortMonth','OrderMonth']).agg({'CustomerID':'nunique'}).reset_index()
cohort['CohortIndex'] = (cohort['OrderMonth'] - cohort['CohortMonth']).apply(lambda x: x.n)
cohort_pivot = cohort.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')
cohort_size = cohort_pivot.iloc[:,0]
retention = cohort_pivot.divide(cohort_size, axis=0)

# Show first 12 cohorts x 6 months retention heatmap
retention_display = retention.iloc[:12, :6].fillna(0)
fig3 = px.imshow(retention_display.values,
                 labels=dict(x='Months since cohort', y='Cohort month', color='Retention'),
                 x=retention_display.columns, y=retention_display.index.astype(str),
                 title='Cohort retention (first 6 months)')
fig3.show()


In [ ]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalAmount': 'sum'
}).rename(columns={'InvoiceDate':'Recency','InvoiceNo':'Frequency','TotalAmount':'Monetary'})

# Filter non-positive monetary
rfm = rfm[rfm['Monetary'] > 0].copy()

# Score into quintiles
rfm['R_score'] = pd.qcut(rfm['Recency'], 5, labels=[5,4,3,2,1])
rfm['F_score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['M_score'] = pd.qcut(rfm['Monetary'], 5, labels=[1,2,3,4,5])
rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

print('RFM head:')
display(rfm.head())

print('\nTop RFM segments (count):')
print(rfm['RFM_score'].value_counts().head(10))

high_value = rfm[(rfm['R_score'].astype(int) >=4) & (rfm['F_score'].astype(int) >=4) & (rfm['M_score'].astype(int) >=4)]
print('\nHigh value customers count:', high_value.shape[0])


RFM head:


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_score
CustomerID,,,,,,,
12347,2,7,4310.00,5,4,5,545
12348,75,4,1797.24,2,3,4,234
12349,19,1,1757.55,4,1,4,414
12350,310,1,334.40,1,1,2,112
12352,36,11,1545.41,3,5,4,354



Top RFM segments (count):
RFM_score
555    341
111    192
455    170
121    138
112    108
444    107
544     98
211     93
122     91
233     81
Name: count, dtype: int64

High value customers count: 946


In [ ]:
rfm_clust = rfm[['Recency','Frequency','Monetary']].copy()
rfm_clust = rfm_clust.replace([np.inf, -np.inf], np.nan).fillna(0)
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_clust)

# choose k via silhouette for k in 2..7
sil = {}
for k in range(2,8):
    km = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = km.fit_predict(rfm_scaled)
    sil[k] = silhouette_score(rfm_scaled, labels)

print('Silhouette scores by k:', sil)
best_k = max(sil, key=sil.get)
print('Best k by silhouette:', best_k)

km = KMeans(n_clusters=best_k, random_state=42, n_init='auto')
rfm['cluster'] = km.fit_predict(rfm_scaled)

cluster_summary = rfm.groupby('cluster').agg({
    'Recency':'median', 'Frequency':'median', 'Monetary':'median', 'RFM_score':'count'
}).rename(columns={'RFM_score':'count'}).reset_index()
print('Cluster summary:')
display(cluster_summary)


Silhouette scores by k: {2: np.float64(0.5575771081186841), 3: np.float64(0.5893483368693321), 4: np.float64(0.6151781898473841), 5: np.float64(0.6033089965311299), 6: np.float64(0.5113827676126673), 7: np.float64(0.5179193508734034)}
Best k by silhouette: 4
Cluster summary:


,cluster,Recency,Frequency,Monetary,count
0,0,240.0,1.0,304.25,1055
1,1,31.0,3.0,810.45,3063
2,2,2.0,76.0,113214.59,11
3,3,4.0,24.0,7904.28,193


In [ ]:
from math import asin, sqrt
def sample_size_for_uplift(baseline_p, lift_absolute, power=0.8, alpha=0.05):
    p1 = baseline_p
    p2 = baseline_p + lift_absolute
    p2 = max(min(p2, 0.999), 0.001)
    h = 2 * (asin(sqrt(p1)) - asin(sqrt(p2)))
    analysis = NormalIndPower()
    n = analysis.solve_power(effect_size=abs(h), power=power, alpha=alpha, alternative='two-sided')
    return int(np.ceil(n))

# Example
baseline = 0.02  # 2% conversion
lift_abs = 0.005 # +0.5 percentage points
n_req = sample_size_for_uplift(baseline, lift_abs)
print(f'For baseline {baseline:.2%} and uplift {lift_abs:.3%}, n per group ≈ {n_req}')


For baseline 2.00% and uplift 0.500%, n per group ≈ 13768


In [ ]:
# Revenue trend
fig = go.Figure()
fig.add_trace(go.Scatter(x=revenue_m['InvoiceDate'], y=revenue_m['TotalAmount'], mode='lines+markers', name='Revenue'))
fig.update_layout(title='Monthly Revenue (interactive)')
fig.show()

# Top RFM segments pie
top_segments = rfm['RFM_score'].value_counts().nlargest(8).reset_index()
top_segments.columns = ['RFM','count']
fig2 = px.pie(top_segments, names='RFM', values='count', title='Top RFM segments (by count)')
fig2.show()

# Cluster counts bar
clust_counts = rfm['cluster'].value_counts().reset_index()
clust_counts.columns = ['cluster','count']
fig3 = px.bar(clust_counts, x='cluster', y='count', title='Cluster sizes')
fig3.show()


In [ ]:
OUT_DIR = '/content/drive/MyDrive/meesho_project_outputs'
os.makedirs(OUT_DIR, exist_ok=True)
rfm.to_parquet(os.path.join(OUT_DIR, 'rfm_customers.parquet'))
rfm.to_csv(os.path.join(OUT_DIR, 'rfm_customers.csv'), index=True)
print('Exported RFM results to', OUT_DIR)

sample_out = df.sample(min(2000, len(df)), random_state=42)
sample_out.to_csv(os.path.join(OUT_DIR, 'sample_orders.csv'), index=False)
print('Exported sample orders for quick visuals.')


Exported RFM results to /content/drive/MyDrive/meesho_project_outputs
Exported sample orders for quick visuals.


In [ ]:
import plotly.subplots as sp

# Create subplots dashboard
fig = sp.make_subplots(
    rows=2, cols=2,
    subplot_titles=("Monthly Revenue", "Monthly Orders", "Top RFM Segments", "Cluster Sizes"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "pie"}, {"type": "bar"}]]
)

# Monthly Revenue
fig.add_trace(go.Scatter(x=revenue_m['InvoiceDate'],
                         y=revenue_m['TotalAmount'],
                         mode='lines',
                         name='Revenue'),
              row=1, col=1)

# Monthly Orders
fig.add_trace(go.Scatter(x=orders_m['InvoiceDate'],
                         y=orders_m['Orders'],
                         mode='lines',
                         name='Orders'),
              row=1, col=2)

# Top RFM segments pie
top_segments = rfm['RFM_score'].value_counts().nlargest(8).reset_index()
top_segments.columns = ['RFM','count']
fig.add_trace(go.Pie(labels=top_segments['RFM'],
                     values=top_segments['count'],
                     name='RFM Segments'),
              row=2, col=1)

# Cluster sizes bar
clusters = rfm['cluster'].value_counts().reset_index()
clusters.columns = ['cluster','count']
fig.add_trace(go.Bar(x=clusters['cluster'],
                     y=clusters['count'],
                     name='Cluster Sizes'),
              row=2, col=2)

fig.update_layout(height=800, width=1000, title="User Growth & Engagement Dashboard (Colab Interactive)")
fig.show()
